In [1]:
import os 
from deep_data_research.commons.utils import pmap_multi
from openai import OpenAI
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import init_chat_model


In [2]:
API_SECRET_KEY = "sk-di6EQiNDxyGzzoFa93389fA755B14eAfAf31B863F2E6C369"
BASE_URL = "https://aihubmix.com/v1"
os.environ["OPENAI_API_KEY"] = API_SECRET_KEY
os.environ["OPENAI_BASE_URL"] = BASE_URL
os.environ["GOOGLE_API_KEY"] = API_SECRET_KEY
os.environ["GOOGLE_BASE_URL"] = BASE_URL

In [3]:
def translate_english_into_chinese(text):
    client = OpenAI(base_url=BASE_URL, api_key=API_SECRET_KEY)
    response = client.chat.completions.create(messages=[{"role": "user", "content": 'Please translate the following english text into chinese:\n' + text}], 
                                                        model='gemini-2.0-flash', temperature=0)
    output = response.choices[0].message.content
    return output


In [4]:
data_dir = 'MinerU_output/NAR_cetergory/RNA sequence databases'
pdf_dirs = os.listdir(data_dir)

texts = []
for pdf_dir in pdf_dirs:
    text_md = os.path.join(data_dir, pdf_dir, 'auto', f'{pdf_dir}.md')
    with open(text_md, 'r') as f:
        text = f.read()
    texts.append(text)

outputs = pmap_multi(translate_english_into_chinese, zip(texts), n_jobs=256, desc='translating chinese to english')

translating chinese to english: 0it [00:00, ?it/s][Parallel(n_jobs=256)]: Using backend LokyBackend with 256 concurrent workers.
translating chinese to english: 10it [00:00, 600.53it/s]
[Parallel(n_jobs=256)]: Done   5 out of  10 | elapsed:   59.9s remaining:   59.9s
[Parallel(n_jobs=256)]: Done  10 out of  10 | elapsed:  1.3min finished


In [5]:
for pdf_dir, output in zip(pdf_dirs, outputs):
    text_cn_md = os.path.join(data_dir, pdf_dir, 'auto', f'{pdf_dir}_cn.md')
    with open(text_cn_md, 'w') as f:
        f.write(output)

In [9]:
def summary_chinese_content(text):
    client = OpenAI(base_url=BASE_URL, api_key=API_SECRET_KEY)
    response = client.chat.completions.create(messages=[{"role": "user", "content": '下面是一个关于数据集的论文，请你帮我总结论文，以下面的形式\n任务目标：<构建这个数据集的目的是什么，为什么要构建这个数据集>\n详细解释：<详细解释任务目标，帮助我们更加的理解，包括对一些会出现的科学概念的解释>\n构建过程：<具体的构建步骤>，目的是什么，输入是什么，输出是什么，如何实现的(用的什么工具，什么操作)>\n下面是论文具体的内容：' + text}], 
                                                        model='gemini-2.5-flash-lite-preview-06-17', temperature=0)
    output = response.choices[0].message.content
    return output

In [10]:
pdf_dirs = os.listdir(data_dir)
texts = []
for pdf_dir in pdf_dirs:
    text_md = os.path.join(data_dir, pdf_dir, 'auto', f'{pdf_dir}_cn.md')
    with open(text_md, 'r') as f:
        text = f.read()
    texts.append(text)

outputs = pmap_multi(summary_chinese_content, zip(texts), n_jobs=256, desc='summary chinese content')

summary chinese content: 0it [00:00, ?it/s][Parallel(n_jobs=256)]: Using backend LokyBackend with 256 concurrent workers.
summary chinese content: 9it [00:00, 7980.71it/s]
[Parallel(n_jobs=256)]: Done   9 out of   9 | elapsed:  3.5min finished


In [11]:
for pdf_dir, output in zip(pdf_dirs, outputs):
    text_cn_md = os.path.join(data_dir, pdf_dir, 'auto', f'{pdf_dir}_cn_summary.md')
    with open(text_cn_md, 'w') as f:
        f.write(output)